## Random Forest Regression

### Setup

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import scipy.stats as stats
import math

!pip install sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [3]:
X_train_full = pd.read_csv("X_train.csv")
X_train_full.head(5)

,High school graduation raw value,Unemployment raw value,Some college raw value,Ratio of population to mental health providers,Median household income raw value,Average Temperature,January Average Temperature,February Average Temperature,March Average Temperature,April Average Temperature,...,April Average Precipitation,May Average Precipitation,June Average Precipitation,July Average Precipitation,August Average Precipitation,September Average Precipitation,October Average Precipitation,November Average Precipitation,December Average Precipitation,RUCC
0,0.847000,0.096000,0.568000,4885.000000,60735.0,54.808333,26.6,36.3,44.2,55.8,...,11.01,5.75,6.53,2.99,2.16,5.30,3.10,5.95,4.90,1.0
1,0.780000,0.066812,0.547429,954.495385,42945.0,74.058333,59.9,61.5,71.2,72.5,...,1.64,7.55,8.57,3.95,6.58,8.25,3.81,0.24,1.01,1.0
2,0.780000,0.069824,0.453978,2573.647059,40994.0,60.825000,38.5,42.9,55.2,59.1,...,2.69,3.51,2.22,3.80,5.73,0.96,0.54,1.99,3.87,6.0
3,0.866534,0.059000,0.785187,4209.193548,81586.0,43.408333,16.3,17.3,24.8,39.0,...,5.51,5.31,5.99,3.51,1.34,1.62,3.07,0.53,1.26,1.0
4,0.806995,0.102000,0.681151,4158.038961,43863.0,60.991667,46.3,44.2,46.8,61.4,...,4.63,3.10,10.77,8.40,5.02,1.72,1.12,2.85,4.85,2.0


In [4]:
Y_train_full = pd.read_csv("Y_train.csv")
Y_train_full.head(5)

,Poor mental health days raw value,Crude Rate
0,4.1,16.6
1,4.2,11.0
2,3.9,24.9
3,2.3,8.7
4,4.0,16.8


### Random Forest regression for predicting Poor Mental Health Days

In [5]:
import time

OUTER_FOLDS = 10
INNER_FOLDS = 5
N_REPEATS = 3

# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

allMAEs = []
allRMSEs = []
allR2s = []

avgKFolds = RepeatedKFold(n_splits=OUTER_FOLDS, n_repeats=N_REPEATS, random_state=1)
counter = 0
# outer cross validation is to get the average metrics across random folds in
# order to select best model to use
for train_index, test_index in avgKFolds.split(X_train_full):
  X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[test_index]
  Y_train, Y_val = Y_train_full.iloc[train_index], Y_train_full.iloc[test_index]
  
  # inner cross validation is for the best hyperparameter
  cv = RepeatedKFold(n_splits=INNER_FOLDS, n_repeats=1, random_state=1)
  model = RandomForestRegressor(n_estimators = 1000, random_state = 42)

  grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                              cv = cv, n_jobs = -1, verbose = 2)
  
  tic = time.perf_counter()
  # col 0 is Poor Mental Health Days
  grid_search.fit(X_train, Y_train.iloc[:,0]) 

  toc = time.perf_counter() 
  print(f"Downloaded the tutorial in {toc - tic:0.4f} seconds")

  predictions = grid_search.predict(X_val)
  mae = mean_absolute_error(Y_val.iloc[:,0], predictions)
  print("mae = " + str(mae))
  rmse = math.sqrt(mean_squared_error(Y_val.iloc[:,0], predictions))
  print("rmse = " + str(rmse))
  r2 = r2_score(Y_val.iloc[:,0], predictions)
  print("r2 = " + str(r2))

  allMAEs.append(mae)
  allRMSEs.append(rmse)
  allR2s.append(r2)
  counter+=1
  print(counter)
    
print(counter)



  

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   49.4s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  9.4min finished


Downloaded the tutorial in 571.9111 seconds
mae = 0.3725401294811723
rmse = 0.5148437867726245
r2 = 0.37816963062589204
1
Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   51.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  9.9min finished


Downloaded the tutorial in 600.9826 seconds
mae = 0.37557789990190155
rmse = 0.49295848204735054
r2 = 0.4228184612534954
2
Fitting 5 folds for each of 288 candidates, totalling 1440 fits


...

Downloaded the tutorial in 561.1825 seconds
mae = 0.37180619117741065
rmse = 0.48849258298059756
r2 = 0.4326872409716427
28
Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   49.1s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  9.2min finished


Downloaded the tutorial in 559.9973 seconds
mae = 0.3666840761769151
rmse = 0.5124680644328299
r2 = 0.4305821342474945
29
Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.9min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  9.2min finished


Downloaded the tutorial in 556.9733 seconds
mae = 0.368972914596546
rmse = 0.50266884007717
r2 = 0.37056462292727965
30
30


In [6]:
averageMAE = sum(allMAEs)/(OUTER_FOLDS * N_REPEATS)
averageRMSE = sum(allRMSEs)/(OUTER_FOLDS * N_REPEATS)
averageR2 = sum(allR2s)/(OUTER_FOLDS * N_REPEATS)


# Make sure to change this model number for each of the different models. It
# just dictates which row in the csv to put the data. 
model_number = 8

full_metrics_df = pd.read_csv("All_Model_Metrics.csv")
full_metrics_df.loc[model_number] = ["Random Forest Regression Poor Mental Health Days", averageMAE, averageRMSE, averageR2]
full_metrics_df.to_csv('All_Model_Metrics.csv', index=False)

### Random Forest regression for predicting Suicides

In [8]:
OUTER_FOLDS = 10
INNER_FOLDS = 5
N_REPEATS = 3

allMAEs = []
allRMSEs = []
allR2s = []

avgKFolds = RepeatedKFold(n_splits=OUTER_FOLDS, n_repeats=N_REPEATS, random_state=1)

# outer cross validation is to get the average metrics across random folds in
# order to select best model to use
for train_index, test_index in avgKFolds.split(X_train_full):
  X_train, X_val = X_train_full.iloc[train_index], X_train_full.iloc[test_index]
  Y_train, Y_val = Y_train_full.iloc[train_index], Y_train_full.iloc[test_index]
    
  # inner cross validation is for the best hyperparameter
  cv = RepeatedKFold(n_splits=INNER_FOLDS, n_repeats=1, random_state=1)
  model = RandomForestRegressor(n_estimators = 1000, random_state = 42)

  grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                              cv = cv, n_jobs = -1, verbose = 2)
  
  tic = time.perf_counter()
  # col 0 is Poor Mental Health Days
  grid_search.fit(X_train, Y_train.iloc[:,1]) 

  predictions = grid_search.predict(X_val)

  mae = mean_absolute_error(Y_val.iloc[:,1], predictions)
  print("mae = " + str(mae))
  rmse = math.sqrt(mean_squared_error(Y_val.iloc[:,1], predictions))
  print("rmse = " + str(rmse))
  r2 = r2_score(Y_val.iloc[:,1], predictions)
  print("r2 = " + str(r2))

  allMAEs.append(mae)
  allRMSEs.append(rmse)
  allR2s.append(r2)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    8.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   53.5s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 10.0min finished


mae = 4.0946531741494505
rmse = 6.301813317855992
r2 = 0.44276729150060046
Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   53.5s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 10.0min finished


mae = 3.5267230971529115
rmse = 4.9058418177252685
r2 = 0.5000966412402603
Fitting 5 folds for each of 288 candidates, totalling 1440 fits


...

mae = 4.1505845754175095
rmse = 5.863433393835188
r2 = 0.4870760313459499
Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   53.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 10.0min finished


mae = 3.7980613478357133
rmse = 5.119913508828482
r2 = 0.4799005489061682
Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   52.4s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed: 10.0min finished


mae = 4.062002284978816
rmse = 5.41375164231074
r2 = 0.4750525183303177


In [9]:
averageMAE = sum(allMAEs)/(OUTER_FOLDS * N_REPEATS)
averageRMSE = sum(allRMSEs)/(OUTER_FOLDS * N_REPEATS)
averageR2 = sum(allR2s)/(OUTER_FOLDS * N_REPEATS)


# Make sure to change this model number for each of the different models. It
# just dictates which row in the csv to put the data. 
model_number = 9

full_metrics_df = pd.read_csv("All_Model_Metrics.csv")
full_metrics_df.loc[model_number] = ["Random Forest Regression Suicides", averageMAE, averageRMSE, averageR2]
full_metrics_df.to_csv('All_Model_Metrics.csv', index=False)